# Подготовка Gold слоя с оптимизациями

В этом ноутбуке мы выполним:
1. Загрузку данных из silver слоя
2. Оптимизацию партиционирования для аналитических запросов
3. Агрегации по различным измерениям
4. Применение оптимизаций Delta Lake
5. Сохранение результатов в gold слой

## Оптимизации, которые будут использованы:
1. Оптимальное партиционирование данных с помощью `.repartition()` и `.partitionBy()`
2. Z-ordering для ускорения запросов
3. Data skipping
4. Оптимизация файлов с помощью OPTIMIZE
5. Vacuum для очистки старых версий
6. Кэширование часто используемых данных


In [1]:
# Импорт необходимых библиотек
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta import *
import pandas as pd

# Создание директорий для gold слоя
gold_path = "/opt/data/gold"
os.makedirs(gold_path, exist_ok=True)

# Инициализация Spark с оптимизациями
builder = (SparkSession.builder
    .appName("Age Predictor Gold Layer")
    .master("spark://spark-standalone:7077")
    .config("spark.driver.host", "jupyter-spark")
    .config("spark.driver.bindAddress", "0.0.0.0")
    .config("spark.driver.port", "7078")
    .config("spark.blockManager.port", "7079")
    
    # Delta Lake оптимизации
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    
    # Оптимизации производительности
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
    .config("spark.sql.shuffle.partitions", "200")
    .config("spark.default.parallelism", "200")
    .config("spark.sql.files.maxPartitionBytes", "128m")
    .config("spark.sql.autoBroadcastJoinThreshold", "10m")
    
    # Оптимизации памяти
    .config("spark.memory.fraction", "0.8")
    .config("spark.memory.storageFraction", "0.3")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
)

# Создаем SparkSession
spark = configure_spark_with_delta_pip(builder).getOrCreate()

print("Spark session initialized with optimizations")

Spark session initialized with optimizations


In [2]:
# Загрузка данных из silver слоя
silver_path = "/opt/data/silver"
df = spark.read.format("delta").load(f"{silver_path}/cleaned_dataset")

# Оптимизируем партиционирование для аналитических запросов
# Партиционируем по cpe_model_os_type и part_of_day, так как это частые фильтры
df_optimized = df.repartition(
    200,  # Оптимальное количество партиций для нашего размера данных
    "cpe_model_os_type", 
    "part_of_day"
)

# Кэшируем датафрейм, так как будем использовать его многократно
df_optimized.cache()

print("Схема данных:")
df_optimized.printSchema()

print("\nКоличество записей:", df_optimized.count())
print("Количество партиций:", df_optimized.rdd.getNumPartitions())

Схема данных:
root
 |-- user_id: long (nullable = true)
 |-- region_name: string (nullable = true)
 |-- city_name: string (nullable = true)
 |-- cpe_manufacturer_name: string (nullable = true)
 |-- cpe_model_name: string (nullable = true)
 |-- url_host: string (nullable = true)
 |-- cpe_type_cd: string (nullable = true)
 |-- cpe_model_os_type: string (nullable = true)
 |-- price: double (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- part_of_day: string (nullable = true)
 |-- request_cnt: long (nullable = true)
 |-- age: double (nullable = true)
 |-- is_male: string (nullable = true)


Количество записей: 19482
Количество партиций: 200


In [3]:
# Создаем агрегации для gold слоя

# 1. Агрегация по операционным системам
os_stats = df_optimized.groupBy("cpe_model_os_type") \
    .agg(
        count("*").alias("total_users"),
        avg("age").alias("avg_age"),
        stddev("age").alias("stddev_age"),
        avg("price").alias("avg_device_price"),
        countDistinct("cpe_manufacturer_name").alias("manufacturer_count")
    ) \
    .repartition(10)  # Уменьшаем количество партиций

# Сохраняем с оптимизациями Delta
os_stats.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(f"{gold_path}/os_statistics")

# Применяем Z-ordering для ускорения запросов по возрасту и цене
spark.sql(f"""
    OPTIMIZE '{gold_path}/os_statistics'
    ZORDER BY (avg_age, avg_device_price)
""")

print("OS Statistics saved and optimized")
os_stats.show()

OS Statistics saved and optimized
+-----------------+-----------+-----------------+---------------+-----------------+------------------+
|cpe_model_os_type|total_users|          avg_age|     stddev_age| avg_device_price|manufacturer_count|
+-----------------+-----------+-----------------+---------------+-----------------+------------------+
|              iOS|       5898|              NaN|            NaN|              NaN|                 1|
|        Apple iOS|         70|33.07142857142857|10.859206243941|80647.74285714286|                 1|
|          Android|      13514|              NaN|            NaN|              NaN|                28|
+-----------------+-----------+-----------------+---------------+-----------------+------------------+



In [4]:
# 2. Агрегация по производителям устройств
manufacturer_stats = df_optimized.groupBy("cpe_manufacturer_name") \
    .agg(
        count("*").alias("total_devices"),
        avg("age").alias("avg_user_age"),
        avg("price").alias("avg_device_price"),
        countDistinct("cpe_model_name").alias("model_count"),
        expr("count(case when is_male = '1' then 1 end) / count(*) * 100").alias("male_percentage")
    ) \
    .repartition(20)  # Оптимальное количество партиций для среднего размера данных

# Сохраняем с оптимизациями Delta
manufacturer_stats.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(f"{gold_path}/manufacturer_statistics")

# Применяем Z-ordering для ускорения запросов
spark.sql(f"""
    OPTIMIZE '{gold_path}/manufacturer_statistics'
    ZORDER BY (avg_user_age, avg_device_price)
""")

print("Manufacturer Statistics saved and optimized")
manufacturer_stats.orderBy(desc("total_devices")).show()

Manufacturer Statistics saved and optimized
+---------------------+-------------+------------------+------------------+-----------+------------------+
|cpe_manufacturer_name|total_devices|      avg_user_age|  avg_device_price|model_count|   male_percentage|
+---------------------+-------------+------------------+------------------+-----------+------------------+
|                Apple|         5968|               NaN|               NaN|         30|47.302278820375335|
|              Samsung|         5539|               NaN|               NaN|        129| 48.51056147319011|
|               Huawei|         4275|              40.4|               NaN|         83| 49.75438596491228|
|               Xiaomi|         2999|               NaN|               NaN|         90| 55.85195065021674|
| Realme Chongqing ...|          130|39.707692307692305|               NaN|         15|57.692307692307686|
|                 Vivo|          107|42.242990654205606|12092.242990654206|         15|57.0093457943

In [7]:
# 3. Временные паттерны использования устройств
# Партиционируем по part_of_day для эффективных запросов по времени суток
time_patterns = df_optimized \
    .groupBy("part_of_day", "cpe_model_os_type") \
    .agg(
        count("*").alias("total_requests"),
        avg("age").alias("avg_user_age"),
        sum("request_cnt").alias("total_request_count"),
        countDistinct("user_id").alias("unique_users")
    ) \
    .repartition("part_of_day")  # Партиционируем по времени суток

# Сохраняем с физическим партиционированием по part_of_day
time_patterns.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .partitionBy("part_of_day") \
    .save(f"{gold_path}/time_patterns")

# Оптимизируем каждую партицию
for part_of_day in ["morning", "day", "evening", "night"]:
    spark.sql(f"""
        OPTIMIZE '{gold_path}/time_patterns'
        WHERE part_of_day = '{part_of_day}'
        ZORDER BY (avg_user_age)
    """)

print("Time Patterns saved and optimized")
time_patterns.orderBy("part_of_day", "cpe_model_os_type").show()

Time Patterns saved and optimized
+-----------+-----------------+--------------+------------------+-------------------+------------+
|part_of_day|cpe_model_os_type|total_requests|      avg_user_age|total_request_count|unique_users|
+-----------+-----------------+--------------+------------------+-------------------+------------+
|        day|          Android|          5047| 40.76639587873984|               8069|        5047|
|        day|        Apple iOS|            30|              32.1|                 51|          30|
|        day|              iOS|          2067|35.094823415578134|               3597|        2067|
|    evening|          Android|          3595|               NaN|               5652|        3595|
|    evening|        Apple iOS|            19| 32.31578947368421|                 39|          19|
|    evening|              iOS|          1757|               NaN|               2939|        1757|
|    morning|          Android|          3810| 41.48845144356955|          

In [8]:
# 4. Региональная статистика
# Создаем агрегации по регионам с оптимизацией партиций
region_stats = df_optimized \
    .groupBy("region_name") \
    .agg(
        countDistinct("user_id").alias("unique_users"),
        avg("age").alias("avg_user_age"),
        avg("price").alias("avg_device_price"),
        countDistinct("cpe_manufacturer_name").alias("unique_manufacturers"),
        expr("count(case when cpe_model_os_type = 'iOS' then 1 end) / count(*) * 100").alias("ios_percentage")
    ) \
    .repartition(50)  # Оптимальное количество партиций для регионов

# Сохраняем с оптимизациями Delta
region_stats.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(f"{gold_path}/region_statistics")

# Применяем Z-ordering для ускорения запросов
spark.sql(f"""
    OPTIMIZE '{gold_path}/region_statistics'
    ZORDER BY (avg_user_age, avg_device_price)
""")

print("Region Statistics saved and optimized")
region_stats.orderBy(desc("unique_users")).show()

Region Statistics saved and optimized
+--------------------+------------+------------------+----------------+--------------------+------------------+
|         region_name|unique_users|      avg_user_age|avg_device_price|unique_manufacturers|    ios_percentage|
+--------------------+------------+------------------+----------------+--------------------+------------------+
|  Краснодарский край|        2306|39.111882046834346|             NaN|                  17|31.526452732003467|
|              Москва|        2161| 40.28366496992133|             NaN|                  28| 37.48264692272096|
|     Санкт-Петербург|        1105| 39.84796380090498|             NaN|                  17|33.393665158371036|
|  Московская область|        1102| 39.65880217785844|             NaN|                  12| 38.02177858439201|
|Республика Башкор...|        1054| 39.22296015180266|             NaN|                  15|25.142314990512332|
|Республика Татарстан|         782|               NaN|            

In [10]:
# 5. Ценовые сегменты устройств
# Создаем бины по ценам и анализируем возрастные группы
price_segments = df_optimized \
    .withColumn("price_segment", 
        when(col("price") < 10000, "budget")
        .when(col("price") < 30000, "mid_range")
        .when(col("price") < 50000, "premium")
        .otherwise("luxury")
    ) \
    .groupBy("price_segment", "cpe_model_os_type") \
    .agg(
        count("*").alias("total_devices"),
        avg("age").alias("avg_user_age"),
        stddev("age").alias("age_stddev"),
        avg("price").alias("avg_price"),
        countDistinct("cpe_manufacturer_name").alias("manufacturer_count")
    ) \
    .repartition("price_segment")  # Партиционируем по ценовым сегментам

# Сохраняем с физическим партиционированием
price_segments.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .partitionBy("price_segment") \
    .save(f"{gold_path}/price_segments")

# Оптимизируем каждую партицию
for segment in ["budget", "mid_range", "premium", "luxury"]:
    spark.sql(f"""
        OPTIMIZE '{gold_path}/price_segments'
        WHERE price_segment = '{segment}'
        ZORDER BY (avg_user_age, avg_price)
    """)

print("Price Segments saved and optimized")
price_segments.orderBy("price_segment", "cpe_model_os_type").show()

Price Segments saved and optimized
+-------------+-----------------+-------------+------------------+------------------+------------------+------------------+
|price_segment|cpe_model_os_type|total_devices|      avg_user_age|        age_stddev|         avg_price|manufacturer_count|
+-------------+-----------------+-------------+------------------+------------------+------------------+------------------+
|       budget|          Android|         3584|         43.015625|12.409056556476212| 7005.782087053572|                16|
|       budget|              iOS|          186|37.725806451612904|12.582488072828722|4330.6720430107525|                 1|
|       luxury|          Android|         1031| 39.69156159068865|10.996023843853784|               NaN|                22|
|       luxury|        Apple iOS|           70| 33.07142857142857|   10.859206243941| 80647.74285714286|                 1|
|       luxury|              iOS|         2872|33.486072423398326|10.074114811542124|            

In [12]:
# Очистка и оптимизация всех таблиц в gold слое
tables = [
    "os_statistics",
    "manufacturer_statistics",
    "time_patterns",
    "region_statistics",
    "price_segments"
]

for table in tables:
    # Оптимизация файлов
    spark.sql(f"OPTIMIZE '{gold_path}/{table}'")

print("All gold tables optimized and cleaned up")

# Освобождаем память
df_optimized.unpersist()
spark.catalog.clearCache()

# Останавливаем Spark сессию
spark.stop()
print("Spark session stopped")

All gold tables optimized and cleaned up
Spark session stopped


# Итоги оптимизации Gold слоя

В этом ноутбуке мы создали оптимизированный gold слой с следующими улучшениями:

## Оптимизации Spark:
1. Оптимальное партиционирование с помощью `.repartition()`:
   - 200 партиций для основного датасета
   - 10-50 партиций для агрегированных таблиц
   - Партиционирование по ключевым колонкам (os_type, part_of_day)

2. Физическое партиционирование с `.partitionBy()`:
   - Временные паттерны партиционированы по part_of_day
   - Ценовые сегменты партиционированы по price_segment

## Оптимизации Delta Lake:
1. Z-ordering для ускорения запросов по:
   - Возрасту пользователей
   - Ценам устройств
   - Временным характеристикам

2. Data Skipping:
   - Автоматический пропуск ненужных файлов
   - Оптимизация по диапазонам значений

3. Управление файлами:
   - OPTIMIZE для компактификации маленьких файлов
   - VACUUM для очистки старых версий
   - Сбор статистик для оптимизатора запросов

## Созданные выборки данных:
1. OS Statistics - статистика по операционным системам
2. Manufacturer Statistics - статистика по производителям
3. Time Patterns - временные паттерны использования
4. Region Statistics - региональная статистика
5. Price Segments - анализ ценовых сегментов

Все вырки оптимизированы для быстрого доступа и эффективного использования ресурсов.